In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/anime.csv')

In [4]:
df.fillna(value={'genre': 'Unknown', 'rating': df['rating'].mean()}, inplace=True)

In [5]:
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [6]:
df.isnull().sum()

,0
anime_id,0
name,0
genre,0
type,25
episodes,0
rating,0
members,0


In [7]:
# Convert genres into numerical representations (dummy variables)
genres = df['genre'].str.get_dummies(sep=', ')
anime_features = pd.concat([df['rating'], genres], axis=1)

# Normalize numerical features (ratings)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
anime_features_scaled = scaler.fit_transform(anime_features)
anime_features_normalized = pd.DataFrame(anime_features_scaled, columns=anime_features.columns)


In [8]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_anime(anime_features, target_anime_index, threshold=0.5, top_n=10):
    # Calculate cosine similarity matrix
    similarity_scores = cosine_similarity(anime_features, anime_features)

    # Get similarity scores for the target anime
    target_scores = similarity_scores[target_anime_index]

    # Find indices of animes with similarity scores above the threshold
    similar_anime_indices = [i for i, score in enumerate(target_scores) if score > threshold and i != target_anime_index]

    # Sort indices based on similarity scores
    similar_anime_indices.sort(key=lambda x: target_scores[x], reverse=True)

    # Return top N similar anime
    return similar_anime_indices[:top_n]

# Example usage:
target_anime_index = 0  # Replace with the index of the anime you want to find similar anime for
similar_anime_indices = recommend_anime(anime_features_normalized.values, target_anime_index)
similar_anime_titles = df.iloc[similar_anime_indices]['name']
print(f"Anime similar to {df.iloc[target_anime_index]['name']}:\n{similar_anime_titles}")


Anime similar to Kimi no Na wa.:
5805                          Wind: A Breath of Heart OVA
6394                         Wind: A Breath of Heart (TV)
1111                Aura: Maryuuin Kouga Saigo no Tatakai
208                         Kokoro ga Sakebitagatterunda.
504     Clannad: After Story - Mou Hitotsu no Sekai, K...
1201                       Angel Beats!: Another Epilogue
1435                                           True Tears
1436                   &quot;Bungaku Shoujo&quot; Memoire
1494                                             Harmonie
1631                                  Kimikiss Pure Rouge
Name: name, dtype: object


In [9]:
def evaluate_recommendation_system(actual_indices, recommended_indices):
    intersection = set(actual_indices) & set(recommended_indices)

    if len(recommended_indices) == 0:
        precision = 0
    else:
        precision = len(intersection) / len(recommended_indices)

    if len(actual_indices) == 0:
        recall = 0
    else:
        recall = len(intersection) / len(actual_indices)

    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    return precision, recall, f1_score


In [10]:
def evaluate_recommendation_system(actual_indices, recommended_indices):
    precision = len(set(actual_indices) & set(recommended_indices)) / len(recommended_indices)
    recall = len(set(actual_indices) & set(recommended_indices)) / len(actual_indices)
    f1_score = 2 * (precision * recall) / (precision + recall)
    return precision, recall, f1_score


In [12]:
# Split the dataset (not shown, typically use train_test_split)
train_data = anime_features_normalized.values

# Evaluate using precision, recall, and F1-score
def evaluate_recommendation_system(actual_indices, recommended_indices):
    # Dummy evaluation function, replace with actual evaluation metrics
    intersection = set(actual_indices) & set(recommended_indices)
    precision = len(intersection) / len(recommended_indices) if recommended_indices else 0  # Handle empty recommended list
    recall = len(intersection) / len(actual_indices) if actual_indices else 0        # Handle empty actual list
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0  # Handle zero division
    return precision, recall, f1_score



# Example evaluation:
actual_indices = [10, 20, 30]  # Replace with actual test cases
recommended_indices = recommend_anime(train_data, 5)  # Example recommendation for anime at index 5
precision, recall, f1_score = evaluate_recommendation_system(actual_indices, recommended_indices)
print(f"Precision: {precision}, Recall: {recall}, F1-score: {f1_score}")


Precision: 0.0, Recall: 0.0, F1-score: 0
